In [2]:
import cv2
import numpy as np
import pyrealsense2 as rs

##### 컴퓨터에 연결된 카메라의 index 값 확인 

In [3]:
def cameraIndex(n):
    arr = []
    index = 0
    while index < n:
        cap = cv2.VideoCapture(index)
        if cap.read()[0]:
            width = cap.get(cv2.CAP_PROP_FRAME_WIDTH) # Frame Width
            height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) # Frame Height
            fps = cap.get(cv2.CAP_PROP_FPS) 
            print('camera ' + str(index) + ' width: ' + str(width) + ', height: ' + str(height) + ', fps: ' + str(fps))
            arr.append(index)
            cap.release()
        index += 1
    return arr

In [4]:
out = cameraIndex(5)
print(out)

camera 0 width: 640.0, height: 480.0, fps: 30.0
camera 2 width: 640.0, height: 480.0, fps: 30.0
camera 3 width: 640.0, height: 480.0, fps: 0.0
[0, 2, 3]


In [7]:
# https://github.com/IntelRealSense/librealsense/blob/master/wrappers/python/examples/opencv_viewer_example.py
def setRealSensor(): 
    # Configure depth and color streams
    global pipeline, align, dwidth, dheight, fps, depthIntrinsic
    pipeline = rs.pipeline()
    config = rs.config()

    # Get device product line for setting a supporting resolution
    pipeline_wrapper = rs.pipeline_wrapper(pipeline)
    pipeline_profile = config.resolve(pipeline_wrapper)
    device = pipeline_profile.get_device()
    device_product_line = str(device.get_info(rs.camera_info.product_line))

    found_rgb = False
    for s in device.sensors:
        if s.get_info(rs.camera_info.name) == 'RGB Camera':
            found_rgb = True
            break
    if not found_rgb:
        print("The demo requires Depth camera with Color sensor")
        exit(0)

    config.enable_stream(rs.stream.depth, dwidth, dheight, rs.format.z16, fps)

    if device_product_line == 'L500':
        config.enable_stream(rs.stream.color, 960, 540, rs.format.bgr8, fps)
    else:
        config.enable_stream(rs.stream.color, dwidth, dheight, rs.format.bgr8, fps)

    align = rs.align(rs.stream.depth)
    pipeline.start(config)

    # Get the intrinsics information for calculation of 3D point
    frames = pipeline.wait_for_frames()
    alignedFrames = align.process(frames)
    depth = alignedFrames.get_depth_frame()
    depthIntrinsic = depth.profile.as_video_stream_profile().intrinsics

    
def stopRealSensor():
    global pipeline
    pipeline.stop()


def myRealSensor():
    global pipeline, align, frames, alignedFrames 
    global colorImage, depthImage, alignedColorImage, alignedDepthImage, alignedDepthColormap

    frames = pipeline.wait_for_frames()

    colorFrame = frames.get_color_frame()
    # Convert images to numpy arrays
    colorImage = np.asanyarray(colorFrame.get_data())
    depthFrame = frames.get_depth_frame()
    depthImage = np.asanyarray(depthFrame.get_data())

    alignedFrames = align.process(frames)

    alignedColorFrame = alignedFrames.get_color_frame()
    alignedColorImage = np.asanyarray(alignedColorFrame.get_data())
    alignedDepthFrame = alignedFrames.get_depth_frame()
    alignedDepthImage = np.asanyarray(alignedDepthFrame.get_data())

    # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
    # https://velog.io/@csp213/%ED%99%94%EC%86%8C-%EC%B2%98%EB%A6%ACPixel-Point-Processing
    convertAlignedDepthImage = cv2.convertScaleAbs(alignedDepthImage, alpha=0.03)
    alignedDepthColormap = cv2.applyColorMap(convertAlignedDepthImage, cv2.COLORMAP_JET)


In [8]:
capture, pipeline, align = 0, 0, 0
colorImage, depthImage, alignedColorImage = 0, 0, 0
alignedDepthImage, alignedDepthColormap = 0, 0
dwidth, dheight, fps = 1280, 720, 30


In [9]:
cv2.namedWindow("colorImage", cv2.WINDOW_NORMAL)        
cv2.resizeWindow("colorImage", dwidth, dheight)  
cv2.namedWindow("depthImage", cv2.WINDOW_NORMAL)        
cv2.resizeWindow("depthImage", dwidth, dheight)  
cv2.namedWindow("alignedColorImage", cv2.WINDOW_NORMAL)        
cv2.resizeWindow("alignedColorImage", dwidth, dheight)  
cv2.namedWindow("alignedDepthImage", cv2.WINDOW_NORMAL)        
cv2.resizeWindow("alignedDepthImage", dwidth, dheight)  
cv2.namedWindow("alignedDepthColormap", cv2.WINDOW_NORMAL)        
cv2.resizeWindow("alignedDepthColormap", dwidth, dheight)  

setRealSensor()
while(True):
    myRealSensor()
    cv2.imshow("colorImage", colorImage)
    cv2.imshow("depthImage", depthImage)
    cv2.imshow("alignedColorImage", alignedColorImage)
    cv2.imshow("alignedDepthImage", alignedDepthImage)
    cv2.imshow("alignedDepthColormap", alignedDepthColormap)
    key = cv2.waitKey(1) & 0xFF # delay_time
    if key == ord('q') or key == 27:
        break
stopRealSensor()
cv2.destroyAllWindows()